# Weather feature extraction

In dit notebook worden er features uit de Weather_Condition kolom gehaald. De manier waarop de kolom nu georganiseerd is maakt het moeilijk om een trend in de verschillende soorten weersverschijnselen te vinden. Om deze reden zullen we elk type weersverschijnsel een eigen kolom geven, en sommige types zullen nog een kolom krijgen met subtypes (zoals 'light rain', 'heavy rain', etc.). 

In [1]:
# Import everything for feature extraction and visualisation.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read the csv file with the data
df = pd.read_csv('US_Accidents_June20_wind_corrected.csv')

In [3]:
# Print all unique values in the Weather_Condition column
unique_weather = df['Weather_Condition'].unique()
print(unique_weather)

['Light Rain' 'Overcast' 'Mostly Cloudy' 'Rain' 'Light Snow' 'Haze'
 'Scattered Clouds' 'Partly Cloudy' 'Clear' 'Snow'
 'Light Freezing Drizzle' 'Light Drizzle' 'Fog' 'Shallow Fog' 'Heavy Rain'
 'Light Freezing Rain' 'Cloudy' 'Drizzle' nan 'Light Rain Showers' 'Mist'
 'Smoke' 'Patches of Fog' 'Light Freezing Fog' 'Light Haze'
 'Light Thunderstorms and Rain' 'Thunderstorms and Rain' 'Fair'
 'Volcanic Ash' 'Blowing Sand' 'Blowing Dust / Windy' 'Widespread Dust'
 'Fair / Windy' 'Rain Showers' 'Mostly Cloudy / Windy'
 'Light Rain / Windy' 'Hail' 'Heavy Drizzle' 'Showers in the Vicinity'
 'Thunderstorm' 'Light Rain Shower' 'Light Rain with Thunder'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'T-Storm'
 'Heavy Thunderstorms and Rain' 'Thunder' 'Heavy T-Storm' 'Funnel Cloud'
 'Heavy T-Storm / Windy' 'Blowing Snow' 'Light Thunderstorms and Snow'
 'Heavy Snow' 'Low Drifting Snow' 'Light Ice Pellets' 'Ice Pellets'
 'Squalls' 'N/A Precipitation' 'Cloudy / Windy' 'Light Fog' 'Sand'
 'Snow G

Zoals je kunt zien zijn er veel unieke waardes, echter er zijn ook veel waardes waar overlap in zit. Zo is er de waarde 'Rain', maar ook 'Rain / Windy'. Deze zouden nu als aparte waardes gezien worden, maar in beide gevallen regent het. Wij denken dus dat het goed is om van de verschillende type weersomstandigheden features te maken, zodat het opsporen van een trend met een specifiek weertype makkelijker wordt.

In [4]:
# Filter out NaN values
unique_weather = unique_weather[np.array([isinstance(item, str) for item in unique_weather])]

## Exploratie van de values

In de volgende sectie worden de verschillende unieke waardes onderzocht. Zo wordt er gekeken welke unieke woorden er in de values voorkomen, en welke woorden hiervan types weer (zoals regen) zijn of woorden om het type weer te beschrijven (zoals *lichte* regen). Aan de hand hiervan bepalen we welke feature kolommen er gemaakt en ingevuld moeten worden.

In [5]:
# Make a set of the unique words in the unique values of weather_condition
unique_words = set()

# Iterate through the rows of the Weather_Condition column and extract unique words
for row in df['Weather_Condition']:
    if type(row) == str:
        words = row.split()  # Split the string into words
        unique_words.update(words)  # Add words to the set of unique words

# Print unique words
print("Unique words in Weather_Condition:")
print(unique_words)

Unique words in Weather_Condition:
{'Tornado', 'Freezing', 'Thunderstorm', 'Squalls', 'Rain', 'Funnel', 'Mix', 'Haze', 'Mist', 'with', 'Sleet', 'Patches', 'Overcast', 'Cloudy', 'and', 'Clouds', '/', 'Precipitation', 'Whirls', 'Dust', 'Volcanic', 'Small', 'Thunderstorms', 'Thunder', 'Pellets', 'Sand', 'Drizzle', 'Scattered', 'Blowing', 'Fog', 'Widespread', 'Low', 'Ice', 'Windy', 'Heavy', 'of', 'Mostly', 'Light', 'Partial', 'Cloud', 'Fair', 'in', 'Showers', 'Clear', 'Nearby', 'Shallow', 'Drifting', 'the', 'Hail', 'T-Storm', 'Ash', 'N/A', 'Vicinity', 'Grains', 'Whirlwinds', 'Snow', 'Shower', 'Wintry', 'Partly', 'Smoke'}


In [6]:
# For each unique word, print in which values it occurs
for word in unique_words:
    words_with_word = [word2 for word2 in unique_weather if word in word2]
    print(f'Values containing {word}: {words_with_word}')  

Values containing Tornado: ['Tornado']
Values containing Freezing: ['Light Freezing Drizzle', 'Light Freezing Rain', 'Light Freezing Fog', 'Heavy Freezing Rain', 'Light Freezing Rain / Windy', 'Freezing Rain', 'Freezing Drizzle', 'Heavy Freezing Drizzle', 'Freezing Rain / Windy']
Values containing Thunderstorm: ['Light Thunderstorms and Rain', 'Thunderstorms and Rain', 'Thunderstorm', 'Heavy Thunderstorms and Rain', 'Light Thunderstorms and Snow', 'Heavy Thunderstorms and Snow', 'Heavy Thunderstorms with Small Hail', 'Light Thunderstorm', 'Thunderstorms and Snow']
Values containing Squalls: ['Squalls', 'Squalls / Windy']
Values containing Rain: ['Light Rain', 'Rain', 'Heavy Rain', 'Light Freezing Rain', 'Light Rain Showers', 'Light Thunderstorms and Rain', 'Thunderstorms and Rain', 'Rain Showers', 'Light Rain / Windy', 'Light Rain Shower', 'Light Rain with Thunder', 'Heavy Thunderstorms and Rain', 'Rain / Windy', 'Heavy Rain / Windy', 'Heavy Freezing Rain', 'Heavy Rain Showers', 'Rain 

Hierboven wordt voor ieder uniek woord dat voorkomt in die weather_condition values bekeken in welke unieke values zij voorkomen. Aan de hand daarvan bekijken we of het woord staat voor een type weer, of een beschrijving van het weer. Door dit te bekijken zijn wij tot de conclusie gekomen dat de volgende typen weer voorkomen in de dataset: 'Overcast', 'Volcanic Ash', 'Dust', 'Wintry Mix', 'Fair', 'Clear', 'Hail', 'Thunder', 'Cloud', 'Tornado', 'T-Storm', 'Shower', 'Thunderstorm', 'Mist', 'Snow', 'Smoke', 'Drizzle', 'Haze', 'Windy', 'Squalls', 'Ice', 'Rain', 'Sand', 'Sleet'. Later zullen wij nog bekijken of we een paar van deze typen weer kunnen samenvoegen omdat het erg op elkaar lijkt. 

In [7]:
weather_types = ['Overcast', 'Volcanic Ash', 'Dust', 'Wintry Mix', 'Fair', 'Clear', 'Hail', 'Thunder', 'Cloud', 'Tornado', 'T-Storm', 'Shower', 'Thunderstorm', 'Mist', 'Snow', 'Smoke', 'Drizzle', 'Haze', 'Windy', 'Squalls', 'Ice', 'Rain', 'Sand', 'Sleet']

In [8]:
for word in weather_types:
    words_with_word = [word2 for word2 in unique_weather if word in word2]
    print(f'Values containing {word}: {words_with_word}')  

Values containing Overcast: ['Overcast']
Values containing Volcanic Ash: ['Volcanic Ash']
Values containing Dust: ['Blowing Dust / Windy', 'Widespread Dust', 'Sand / Dust Whirlwinds', 'Blowing Dust', 'Widespread Dust / Windy', 'Sand / Dust Whirlwinds / Windy', 'Dust Whirls', 'Sand / Dust Whirls Nearby']
Values containing Wintry Mix: ['Wintry Mix', 'Wintry Mix / Windy', 'Thunder / Wintry Mix / Windy']
Values containing Fair: ['Fair', 'Fair / Windy']
Values containing Clear: ['Clear']
Values containing Hail: ['Hail', 'Small Hail', 'Light Hail', 'Heavy Thunderstorms with Small Hail', 'Thunder and Hail', 'Thunder and Hail / Windy']
Values containing Thunder: ['Light Thunderstorms and Rain', 'Thunderstorms and Rain', 'Thunderstorm', 'Light Rain with Thunder', 'Thunder in the Vicinity', 'Heavy Thunderstorms and Rain', 'Thunder', 'Light Thunderstorms and Snow', 'Heavy Thunderstorms and Snow', 'Thunder / Windy', 'Light Snow with Thunder', 'Snow and Thunder', 'Heavy Snow with Thunder', 'Thunder

Hier kunnen we zien welke verschillende beschrijvingen er zijn voor de verschillende typen weer. Hierbij rekenen we combinaties zoals 'Light Rain / Windy' voor het type 'Rain' als 'Light Rain', aangezien 'Windy' weer een ander type weer is. 
- Overcast: 'Overcast'
- Volcanic Ash: 'Volcanic Ash'
- Dust: 'Blowing Dust', 'Widespread Dust', 'Dust Whirlwinds', 'Dust Whirls', 'Dust Whirls Nearby'
- Wintry Mix: 'Wintry Mix'
- Fair: 'Fair'
- Clear: 'Clear'
- Hail: 'Hail', 'Small Hail', 'Light Hail'
- Thunder: 'Thunder', 'Thunder in the Vicinity'
- Cloud: 'Mostly Cloudy', 'Scattered Clouds', 'Partly Cloudy', 'Cloudy', 'Funnel Cloud'- Tornado: 'Tornado'
- T-storm: 'T-Storm', 'Heavy T-Storm'
- Shower: 'Showers in the Vicinity' (In de andere gevallen is shower meer een beschrijving)
- Thunderstorm: 'Light Thunderstorms', 'Thunderstorms', 'Thunderstorm', 'Heavy Thunderstorms', 'Light Thunderstorm'
- Mist: 'Mist'
- Snow: 'Light Snow', 'Snow', 'Blowing Snow', 'Heavy Snow', 'Low Drifting Snow', 'Snow Grains', 'Snow Showers', 'Light Snow Shower', 'Light Snow Grains', 'Heavy Blowing Snow', 'Light Blowing Snow', 'Drifting Snow'
- Smoke: 'Smoke', 'Heavy Smoke'
- Drizzle: 'Light Freezing Drizzle', 'Light Drizzle', 'Drizzle', 'Heavy Drizzle, 'Freezing Drizzle', 'Heavy Freezing Drizzle'
- Haze: 'Haze', 'Light Haze'
- Windy: 'Windy'
- Squalls: 'Squalls'
- Ice: 'Light Ice Pellets', 'Ice Pellets', 'Heavy Ice Pellets'
- Rain: 'Light Rain', 'Rain', 'Heavy Rain', 'Light Freezing Rain', 'Light Rain Showers', 'Rain Showers', 'Light Rain Shower', 'Heavy Freezing Rain', 'Heavy Rain Showers', 'Rain Shower', 'Light Rain Shower', 'Freezing Rain', 'Heavy Rain Shower'
- Sand: 'Blowing Sand', 'Sand'
- Sleet: 'Sleet', 'Light Sleet', 'Heavy Sleet'

Sommige weersomstandigheden zou je in een zelfde categorie kunnen plaatsen. Zo is er bij 'Cloud' en 'Overcast' beide sprake van wolken in de lucht. Bij 'Fair' en 'Clear' is er beide sprake van relatief goed weer waarbij er geen sprake is van neerslag of bewolking. 'Shower' is een vorm van regen, waardoor het samen met 'Rain' in een categorie geplaatst zou kunnen worden. Deze categoriën worden dan;
- Cloudy: 'Overcast', 'Mostly Cloudy', 'Scattered Clouds', 'Partly Cloudy', 'Cloudy', 'Funnel Cloud'
- Fair/Clear -> alleen True of False values
- Rain: 'Showers in the Vicinity', 'Light Rain', 'Rain', 'Heavy Rain', 'Light Freezing Rain', 'Light Rain Showers', 'Rain Showers', 'Light Rain Shower', 'Heavy Freezing Rain', 'Heavy Rain Showers', 'Rain Shower', 'Light Rain Shower', 'Freezing Rain', 'Heavy Rain Shower'

De rest van de categoriën zullen wij voor nu zo laten.]

In [9]:
# Make new columns for different weather features
new_weather_features = ['Volcanic Ash', 'Dust', 'Wintry Mix', 'Fair/Clear', 'Hail', 'Thunder', 'Cloudy', 'Tornado', 'T-Storm', 'Thunderstorm', 'Mist', 'Snow', 'Smoke', 'Drizzle', 'Haze', 'Windy', 'Squalls', 'Ice Pellets', 'Rain', 'Sand', 'Sleet']
for weather_condition in new_weather_features:
    df[weather_condition] = np.nan

# Make new columns for weather feature types
new_weather_feature_types = ['Dust_Type', 'Hail_Type', 'Thunder_Type', 'Cloudy_Type', 'T-Storm_Type', 'Thunderstorm_Type', 'Snow_Type', 'Smoke_Type', 'Drizzle_Type', 'Haze_Type', 'Ice-Pellets_Type', 'Rain_Type', 'Sand_Type', 'Sleet_Type']
for weather_type in new_weather_feature_types:
    df[weather_type] = np.nan

In [10]:
# function to fill in boolean value of weather feature
def add_boolean_weather_row(index, value, weatherString):
    if pd.notna(value) and isinstance(value, str) and weatherString in value:
        df.at[index, weatherString] = True
        return True
    elif pd.notna(value) and isinstance(value, str):
        df.at[index, weatherString] = False
        return False
    return None

# function to fill in weather type
def add_weather_type(index, value, weatherStringArray, weatherType):
    for weatherString in weatherStringArray:
        if pd.notna(value) and isinstance(value, str) and weatherString in value:
            df.at[index, weatherType] = weatherString
            break

Nu we de nieuwe kolommen gemaakt hebben en de functies gedeclareerd hebben, kunnen we de kolommen gaan invullen. De samengevoegde kolommen hebben een aangepaste aanpak nodig. De rest van de kolommen zullen volgens een forloop door middel van de gedeclareerde functies ingevuld worden.

In [11]:
# for each row, fill in boolean weather feature columns and weather type (str) columns
for index, row in df.iterrows():
    value = row['Weather_Condition']
    # handle 'Cloudy' column and include 'Overcast'
    if pd.notna(value) and isinstance(value, str) and ('Overcast' in value or 'Cloud' in value):
        df.at[index, 'Cloudy'] = True
        add_weather_type(index, value, ['Overcast', 'Mostly Cloudy', 'Scattered Clouds', 'Partly Cloudy', 'Funnel Cloud', 'Cloudy'], 'Cloudy_Type')
    elif pd.notna(value) and isinstance(value, str):
        df.at[index, 'Cloudy'] = False
        df.at[index, 'Cloudy_Type'] = 'None'

    # handle 'Rain' column and include 'Showers in the Vicinity'
    if pd.notna(value) and isinstance(value, str) and ('Showers in the Vicinity' in value or 'Rain' in value):
        df.at[index, 'Rain'] = True
        add_weather_type(index, value, ['Showers in the Vicinity', 'Heavy Freezing Rain', 'Light Freezing Rain', 'Heavy Rain Showers', 'Light Rain Showers', 'Heavy Rain Shower', 'Light Rain Shower', 'Light Rain', 'Heavy Rain', 'Rain Showers', 'Rain Shower', 'Freezing Rain', 'Rain'], 'Rain_Type')
    elif pd.notna(value) and isinstance(value, str):
        df.at[index, 'Rain'] = False
        df.at[index, 'Rain_Type'] = 'None'

    # handle 'Fair/Clear' column and combine 'Fair' and 'Clear' values
    if pd.notna(value) and isinstance(value, str) and ('Clear' in value or 'Fair' in value):
        df.at[index, 'Fair/Clear'] = True
    elif pd.notna(value) and isinstance(value, str):
        df.at[index, 'Fair/Clear'] = False

    # handle all remaining weather feature columns which have different weather types, and handle their type columns
    weather_features_wtype = ['Dust', 'Hail', 'Thunder', 'T-Storm', 'Thunderstorm', 'Snow', 'Smoke', 'Drizzle', 'Haze', 'Ice Pellets', 'Sand', 'Sleet']
    weather_features_wtype_types = [['Dust Whirls Nearby', 'Blowing Dust', 'Widespread Dust', 'Dust Whirlwinds', 'Dust Whirls'], ['Small Hail', 'Light Hail', 'Hail'], ['Thunder in the Vicinity', 'Thunder'], ['Heavy T-Storm', 'T-Storm'], ['Light Thunderstorms', 'Light Thunderstorm', 'Heavy Thunderstorms', 'Thunderstorms', 'Thunderstorm'], ['Low Drifting Snow', 'Light Snow Shower', 'Light Snow Grains', 'Heavy Blowing Snow', 'Light Blowing Snow', 'Snow Grains', 'Heavy Snow', 'Light Snow', 'Blowing Snow', 'Snow Showers', 'Drifting Snow', 'Snow'], ['Heavy Smoke', 'Smoke'], ['Light Freezing Drizzle', 'Heavy Freezing Drizzle', 'Light Drizzle', 'Heavy Drizzle', 'Freezing Drizzle', 'Drizzle'], ['Light Haze', 'Haze'], ['Light Ice Pellets', 'Heavy Ice Pellets', 'Ice Pellets'], ['Blowing Sand', 'Sand'], ['Light Sleet', 'Heavy Sleet', 'Sleet']]
    weather_features_wtype_tname = ['Dust_Type', 'Hail_Type', 'Thunder_Type', 'T-Storm_Type', 'Thunderstorm_Type', 'Snow_Type', 'Smoke_Type', 'Drizzle_Type', 'Haze_Type', 'Ice-Pellets_Type', 'Sand_Type', 'Sleet_Type']

    for i, condition in enumerate(weather_features_wtype):
        wbool = add_boolean_weather_row(index, value, condition)
        if wbool:
            add_weather_type(index, value, weather_features_wtype_types[i], weather_features_wtype_tname[i])
        elif wbool == False:
            df.at[index, weather_features_wtype_tname[i]] = 'None'

    # handle all remaining weather feature columns which only have boolean values
    weather_features_withoutt = ['Volcanic Ash', 'Wintry Mix', 'Tornado', 'Mist', 'Windy', 'Squalls']

    for condition in weather_features_withoutt:
        add_boolean_weather_row(index, value, condition)      

C:\Users\rache\AppData\Local\Temp\ipykernel_21824\3516974852.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Cloudy'] = False
C:\Users\rache\AppData\Local\Temp\ipykernel_21824\3516974852.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Cloudy_Type'] = 'None'
C:\Users\rache\AppData\Local\Temp\ipykernel_21824\3516974852.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Rain'] = True
C:\Users\rache\AppData\Local\Temp\ipykernel_21824\12569

Nu alle kolommen ingevuld zijn, doen we nog een laatste check om te controlleren of er nu logische waardes in de nieuwe kolommen staan. Dit doen we door middel van het uitprinten van de value counts voor elke kolom, en door te kijken naar het aantal NaN values in elke kolom. Als het goed is zou elke kolom 76138 waardes moeten missen, aangezien de Weather_Conditions kolom ook zoveel waardes mist. Verder zullen we voor elke weather type kolom controlleren of elke unieke waarde die we eerder hebben gezien ook daadwerkelijk minstens 1 keer voorkomt.

In [12]:
# Print value counts of each (boolean) value of the weather feature columns, and check if all non-NaN values have been added to the columns
columns = ['Volcanic Ash', 'Dust', 'Wintry Mix', 'Fair/Clear', 'Hail', 'Thunder', 'Cloudy', 'Tornado', 'T-Storm', 'Thunderstorm', 'Mist', 'Snow', 'Smoke', 'Drizzle', 'Haze', 'Windy', 'Squalls', 'Ice Pellets', 'Rain', 'Sand', 'Sleet']
for column in columns:
    unique_values = df[column].unique()
    unique_values = [value for value in unique_values if value is not None and not pd.isna(value)]
    for value in unique_values:
        print(f"Number of occurrences of {value} in {column}: {df[column].value_counts()[value]}")
    print(f"Total NaN: {df[column].isnull().sum()}")

Number of occurrences of False in Volcanic Ash: 3437457
Number of occurrences of True in Volcanic Ash: 22
Total NaN: 76138
Number of occurrences of False in Dust: 3437107
Number of occurrences of True in Dust: 372
Total NaN: 76138
Number of occurrences of False in Wintry Mix: 3435757
Number of occurrences of True in Wintry Mix: 1722
Total NaN: 76138
Number of occurrences of False in Fair/Clear: 2073602
Number of occurrences of True in Fair/Clear: 1363877
Total NaN: 76138
Number of occurrences of False in Hail: 3437435
Number of occurrences of True in Hail: 44
Total NaN: 76138
Number of occurrences of False in Thunder: 3414854
Number of occurrences of True in Thunder: 22625
Total NaN: 76138
Number of occurrences of False in Cloudy: 1792909
Number of occurrences of True in Cloudy: 1644570
Total NaN: 76138
Number of occurrences of False in Tornado: 3437476
Number of occurrences of True in Tornado: 3
Total NaN: 76138
Number of occurrences of False in T-Storm: 3431184
Number of occurrences 

In [13]:
# Print value counts of each weather type value of the weather type columns, and check if all non-NaN values have been added to the columns
columns = ['Dust_Type', 'Hail_Type', 'Thunder_Type', 'Cloudy_Type', 'T-Storm_Type', 'Thunderstorm_Type', 'Snow_Type', 'Smoke_Type', 'Drizzle_Type', 'Haze_Type', 'Ice-Pellets_Type', 'Rain_Type', 'Sand_Type', 'Sleet_Type']
for column in columns:
    unique_values = df[column].unique()
    unique_values = [value for value in unique_values if value is not None and not pd.isna(value)]
    for value in unique_values:
        print(f"Number of occurrences of {value} in {column}: {df[column].value_counts()[value]}")
    print(f"Total NaN: {df[column].isnull().sum()}")

Number of occurrences of None in Dust_Type: 3437107
Number of occurrences of Blowing Dust in Dust_Type: 190
Number of occurrences of Widespread Dust in Dust_Type: 149
Number of occurrences of Dust Whirlwinds in Dust_Type: 31
Number of occurrences of Dust Whirls in Dust_Type: 1
Number of occurrences of Dust Whirls Nearby in Dust_Type: 1
Total NaN: 76138
Number of occurrences of None in Hail_Type: 3437435
Number of occurrences of Hail in Hail_Type: 4
Number of occurrences of Small Hail in Hail_Type: 37
Number of occurrences of Light Hail in Hail_Type: 3
Total NaN: 76138
Number of occurrences of None in Thunder_Type: 3414854
Number of occurrences of Thunder in Thunder_Type: 19597
Number of occurrences of Thunder in the Vicinity in Thunder_Type: 3028
Total NaN: 76138
Number of occurrences of None in Cloudy_Type: 1792909
Number of occurrences of Overcast in Cloudy_Type: 382485
Number of occurrences of Mostly Cloudy in Cloudy_Type: 492532
Number of occurrences of Scattered Clouds in Cloudy_T

Nu we hebben gezien dat alle 'Weather_Condition' waardes in de verschillende kolommen verwerkt zijn, kunnen we de getransformeerde dataframe opslaan in een nieuwe csv file.

In [14]:
df.to_csv('US_Accidents_June20_weather_corrected.csv', index=False)